In [ ]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pyautogui, os, time

In [ ]:
websites = {
            'NYTimes':'https://www.nytimes.com/',
            'BBC':'https://www.bbc.com/news',
            'Reuters':'https://www.reuters.com/', 
            'TheGuardian':'https://www.theguardian.com/us'
}

In [ ]:
# make sure you have a chromedriver binary in the path below
# or change the path to point it to the binary
chromedriver = "chromedriver/chromedriver_mac"

In [ ]:
os.environ["webdriver.chrome.driver"] = chromedriver

In [ ]:
os.makedirs("Outputs", exist_ok=True)

In [ ]:
for website in websites.keys():
    os.makedirs("Outputs/{}".format(website))

In [ ]:
# fetch ads for a given user 
# br: webbrowser instance
# user: user name
# name: website name
# URL: website URL
def fetch_ads(br, user, name, URL):
    br.get(URL)
    if name == 'BBC':
        br.refresh()
        time.sleep(3)
    else:
        br.get(URL)
    time.sleep(2)
    if name == "TheGuardian":
        js = "var aa=document.getElementsByClassName('js-site-message')[0];aa.parentNode.removeChild(aa)"
        br.execute_script(js)
    ctr = 10
    body = br.find_element_by_css_selector('body')
    time.sleep(2)
    for _ in range(ctr):
        try:
            body.send_keys(Keys.PAGE_DOWN)
        except:
            body = br.find_element_by_css_selector('body')
            body.click()
            body.send_keys(Keys.PAGE_DOWN)
        time.sleep(2)
    time.sleep(4)
    frames = br.find_elements_by_css_selector('iframe[id^=google_ads]')
    for i, frame in enumerate(frames):
        actions = ActionChains(br)
        actions.move_to_element(frame).perform()
        time.sleep(2)
        br.switch_to.frame(frame)
        try:
            body = br.find_elements_by_css_selector('body')[0]
            os.makedirs('Outputs/{}'.format(user), exist_ok=True)
            os.makedirs('Outputs/{}/{}'.format(user, name), exist_ok=True)
            body.screenshot('Outputs/{}/{}/{}_{}.png'.format(user, name, name, i+1))
        except:
            pass
        br.switch_to.default_content()

In [ ]:
# login into google
def login(br, user, password):
    br.get("https://accounts.google.com/signin/v2/identifier?hl=en&passive=true&continue=https%3A%2F%2Fwww.google.com%2F%3Fgws_rd%3Dssl&flowName=GlifWebSignIn&flowEntry=ServiceLogin")
    text = br.find_elements_by_css_selector('input[id=identifierId]')
    text = text[0]
    for letter in user:
        text.send_keys(letter)
    text.send_keys(Keys.ENTER)
    time.sleep(2)
    text = br.find_elements_by_css_selector('input[type=password]')
    text = text[0]
    for letter in password:
        text.send_keys(letter)
    text.send_keys(Keys.ENTER)

In [ ]:
br = webdriver.Chrome(chromedriver)
name = "" # user name goes here
user = "" # user email ID goes here
password = "" # password goes here
login(br, user, password)

In [ ]:
time.sleep(2)
for key in websites.keys():
    fetch_ads(br, user, key, websites[key])